In [1]:
import setup
setup.init_django()

0pEnqPfhRk7bYu5LSa7ppMuKfqr21kck


In [2]:
from market.models import StockQuote

In [3]:
from django.db.models import (
    Avg, 
    F,
    RowRange,
    Window,
    Max,
    Min,
    ExpressionWrapper,
    DecimalField,
    Case,
    When,
    Value
)
from django.db.models.functions import TruncDate, FirstValue, Lag, Coalesce
from django.utils import timezone
from datetime import timedelta
from decimal import Decimal

In [6]:
import pandas as pd
from datetime import timedelta
from decimal import Decimal
from django.utils import timezone

def time_bucket_in_python(data, bucket_size='1D'):
    df = pd.DataFrame(data)  # Convert your query results to a DataFrame
    df['time'] = pd.to_datetime(df['time'])
    df.set_index('time', inplace=True)
    return df.resample(bucket_size).agg({
        'close_price': 'last',  # Use the last close price in each bucket
    }).reset_index()

def calculate_rsi(ticker, period=14):
    """
    Calculate Relative Strength Index (RSI) using Django ORM and pandas.

    Args:
        ticker (str): Stock ticker symbol
        period (int): RSI period (default: 14)

    Returns:
        dict: RSI value and component calculations
    """
    end_date = timezone.now()
    start_date = end_date - timedelta(days=period * 4)

    # Fetch price data from the database
    raw_data = list(
        StockQuote.objects.filter(
            company__ticker=ticker, time__range=(start_date, end_date)
        ).values('time', 'close_price')
    )

    # Perform time bucketing using the helper function
    daily_data = time_bucket_in_python(raw_data, bucket_size='1D')

    # Ensure no None values and cast to a float for calculations
    daily_data['close_price'] = daily_data['close_price'].fillna(0).astype(float)

    # Calculate price changes and gains/losses
    daily_data['price_change'] = daily_data['close_price'].diff()
    daily_data['gain'] = daily_data['price_change'].apply(lambda x: x if x > 0 else 0)
    daily_data['loss'] = daily_data['price_change'].apply(lambda x: -x if x < 0 else 0)

    # Calculate initial averages for gains and losses
    initial_data = daily_data.iloc[:period]
    avg_gain = Decimal(initial_data['gain'].mean())
    avg_loss = Decimal(initial_data['loss'].mean())

    # Calculate EMA-based RSI
    alpha = Decimal(1 / period)  # Smoothing factor

    for index, row in daily_data.iloc[period:].iterrows():
        avg_gain = (avg_gain * (1 - alpha) + Decimal(row['gain']) * alpha)
        avg_loss = (avg_loss * (1 - alpha) + Decimal(row['loss']) * alpha)

    # Prevent division by zero
    if avg_loss == 0:
        rsi = 100
    else:
        rs = avg_gain / avg_loss
        rsi = 100 - (100 / (1 + rs))

    return {
        'rsi': round(float(rsi), 4),
        'avg_gain': round(float(avg_gain), 4),
        'avg_loss': round(float(avg_loss), 4),
        'period': period
    }


In [7]:
rsi_data = calculate_rsi('NVDA')

In [8]:
rsi_data

{'rsi': 52.3131, 'avg_gain': 26.9789, 'avg_loss': 24.5931, 'period': 14}